In [ ]:
import importlib
import datetime
import random

import folium
import haversine
import pandas as pd
import numpy as np
from geopy import distance as gp_distance

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=[],
)

### Assumptions

#### Модель
- Модель описывает только К2 на последней миле, все остальное не учитывается
- Все расстояния считаются по прямой, но скорость курьера - средняя скорость такого типа курьера в сервисе от двери до двери (включая подъем на этаж и тд, т.е. целиком)
- Мы знаем получасовой слот времени доставки для всех доставок (и Лавки и Маркета)
- В нашем мире бесконечный саплай курьеров и невероятная эффективность: каждый раз, когда нужен курьер - он стоит прямо у ПВЗ/Лавки
- Предполагаем, что ПВЗ - то же самое, что лавка (в той же локации), только без заказов лавки
- Место в ПВЗ/Лавках никогда не кончается
- Время доставки для юзера считается как случайное время во временном интервале, который он выбрал

#### Продукт
- Приносим только заказы до 10кг, но объем не учитываем
- SLA доставки (для маркета и лавки одинаковый) - 10 минут
- Если курьер на велосипеде физически не может успеть отвезти посылку за 10 минут от ПВЗ/Лавки - мы не даем возможность заказать такую доставку


#### Supply
- Все курьеры - велосипедисты
- Все курьеры получают деньги за SH - 260р в Мск и 250р в Казани

- Курьер может унести не больше 10кг за раз, может взять не больше 3-х посылок, может быть в пути не больше 4-х часов

#### Батчинг
- Принцип батчинга: если путь от точки Б первой доставки до точки Б второй < SLA и попадает в слот от юзера, можно забатчить такие посылки
- Мы не батчим с другими заказами доставки

### Define functions

In [ ]:
def convert_to_datetime(dttm_str):
    return datetime.datetime.strptime(dttm_str, '%Y-%m-%d %H:%M:%S')

def choose_random_time_in_interval(dttm_left, dttm_right):
    delta = (dttm_right - dttm_left).total_seconds()
    return dttm_left + datetime.timedelta(seconds=random.randint(0, delta))

def parse_gps_str(gps_str):
    return tuple(float(c) for c in gps_str.split(','))

def assign_day(dttm, month, day):
    return dttm.replace(month=month, day=day)

### Get and prepare dataframes

In [ ]:
additional_pp = pd.read_csv('./analytics/real_pp_coordinates_kazan.csv', index_col=0)

In [ ]:
additional_pp.head()

In [ ]:
market_orders_df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/kazan_deliveries_2020')
    .as_dataframe()
)
market_orders_df = market_orders_df[~pd.isnull(market_orders_df['final_delivery_point_gps'])]

In [ ]:
market_orders_df.head(2)

In [ ]:
add_pickup_points_df = pd.DataFrame()
add_pickup_points_df['pickup_point_id'] = 1000 + np.arange(len(additional_pp))
add_pickup_points_df['lat'] = additional_pp['lat'].values
add_pickup_points_df['lon'] = additional_pp['lon'].values
add_pickup_points_df['take_start_tm'] = '06:00:00'
add_pickup_points_df['take_end_tm'] = '07:00:00'
add_pickup_points_df['address'] = None

In [ ]:
pickup_points_df = pd.concat([add_pickup_points_df], axis=0)

In [ ]:
pickup_points_df.head()

In [ ]:
market_orders_df['original_delivery_from_datetime'] = (
    market_orders_df['original_delivery_from_datetime'].apply(
        lambda x: x[:10] + ' ' + '09:00:00' if x[11:] < '09:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 10, 29)
    )
)
market_orders_df['original_delivery_to_datetime'] = (
    market_orders_df['original_delivery_to_datetime'].apply(
        lambda x: x[:10] + ' ' + '23:00:00' if x[11:] > '23:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 10, 29)
    )
)

In [ ]:
# Market Orders.
market_deliveries_df = pd.DataFrame()
market_deliveries_df['service_name'] = ['market']*len(market_orders_df)
market_deliveries_df['weight'] = market_orders_df['order_physical_weight'].fillna(
    np.mean(market_orders_df['order_physical_weight'])
)
market_deliveries_df['volume'] = None
market_deliveries_df['delivery_lat'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[1])
)
market_deliveries_df['delivery_lon'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[0])
)
market_deliveries_df['warehouse_lat'] = None
market_deliveries_df['warehouse_lon'] = None
market_deliveries_df['warehouse_id'] = None
market_deliveries_df['delivery_plan_dttm'] = market_orders_df.apply(
    lambda x: choose_random_time_in_interval(
        x['original_delivery_from_datetime'],
        x['original_delivery_to_datetime']
    ),
    axis=1
)
market_deliveries_df['original_delivery_from_datetime'] = market_orders_df.apply(
    lambda x: assign_day(x['original_delivery_from_datetime'], 10, 29),
    axis=1
)
market_deliveries_df['original_delivery_to_datetime'] = market_orders_df.apply(
    lambda x: assign_day(x['original_delivery_to_datetime'], 10, 29),
    axis=1
)
market_deliveries_df['order_id_source'] = market_orders_df['order_id'].astype(int)
market_deliveries_df['warehouse_ready_dttm'] = None
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lat'])]
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lon'])]

In [ ]:
market_deliveries_df = market_deliveries_df[market_deliveries_df['weight']<=10]

In [ ]:
market_deliveries_df.head()

### Run simulation

In [ ]:
simulation_paramaters = {
    'Market_x0.5': {'frac_lavka_orders': 0., 'frac_market_orders': int(500/2)},
    'Market_x1': {'frac_lavka_orders': 0., 'frac_market_orders': int(500)},
    'Market_x2': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*2)},
    'Market_x3': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*3)},
    'Market_x4': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*4)},
    'Market_x5': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*5)},
    'Market_x6': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*6)},
    'Market_x7': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*7)},
    'Market_x8': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*8)},
    'Market_x9': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*9)},
    'Market_x10': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*10)},
    'Market_x11': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*11)},
    'Market_x12': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*12)},
    'Market_x13': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*13)},
    'Market_x14': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*14)},
    'Market_x15': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*15)},
}

In [ ]:
simulation_dict = {}
for key, value in simulation_paramaters.items():
    simulation_dict[key] = {}
    
    deliveries_df = pd.concat(
        [market_deliveries_df.sample(n=value['frac_market_orders'], random_state=7)],
        axis=0
    )
    print(key)
    print(deliveries_df.shape)
    
    sim = common.Simulation(sla_seconds=20*60, sh_cost=250, pp_work_time=14)
    simulation_dict[key]['simulation'] = sim
    
    sim.initialize_simulation(deliveries_df, pickup_points_df)
    sim.assign_objects()
    sim.batch_deliveries_pp_to_user()
    sim.pay_couriers()
    print('\n')

In [ ]:
stats = {}
for key in simulation_dict.keys():
    
    sim = simulation_dict[key]['simulation']
    stats[(key, 'all')] = sim.calculate_metrics(sim.deliveries)

In [ ]:
pd.DataFrame(stats).T.to_excel(
    './analytics/kazan_sla{sla}min_{pp_type}pp_10pvz.xlsx'
    .format(sla=20, pp_type='real')
)

### Draw orders on map

In [ ]:
sim = simulation_dict['Market_x3']['simulation']

In [ ]:
YA_MAPS_TILES = 'http://vec02.maps.yandex.net/tiles?l=map&v=2.2.3&x={x}&y={y}&z={z}'

In [ ]:
f_map = folium.Map(
    location=(55.8, 49), zoom_start=10, tiles=YA_MAPS_TILES, max_zoom=20,
    attr='My Data Attribution', crs='EPSG3395'
)

In [ ]:
for d in sim.deliveries:
    c = 'red'
    if d.delivered_flg:
        c = 'blue'
    folium.CircleMarker(
        location=(d.location_lat, d.location_lon),
        radius=0.001,
        color=c,
        fill=True,
        weight=0.99,
    ).add_to(f_map)

In [ ]:
f_map.save('./kazan_market_x3.html')

### Get k2 vs ndd data

In [ ]:
sim = simulation_dict['Market_x1']['simulation']

In [ ]:
for pp in sim.pickup_points:
    pp.deliveries = [d for d in sim.deliveries if d.pickup_point_id == pp.id]

In [ ]:
for pp in sim.pickup_points:
    print(pp.id, len([d for d in pp.deliveries if d.delivered_flg]))

In [ ]:
pp = sim.pickup_points_dict[1008]

In [ ]:
oid_set = set([d.order_id_source for d in pp.deliveries if d.delivered_flg])

In [ ]:
orders = market_orders_df[
    market_orders_df['order_id'].isin(oid_set)
][['order_id', 'final_delivery_point_address',
   'original_delivery_from_datetime', 'original_delivery_to_datetime',]]

In [ ]:
orders_excel = pd.DataFrame()
orders_excel['id'] = orders['order_id']
orders_excel['address'] = orders['final_delivery_point_address']
orders_excel['time_window'] = orders.apply(
    lambda x: ' - '.join((str(x['original_delivery_from_datetime'])[11:16],
                          str(x['original_delivery_to_datetime'])[11:16])), axis=1
)
orders_excel['shared_service_duration_s'] = 240
orders_excel['service_duration_s'] = 120

In [ ]:
orders_excel.to_excel('./analytics/k2_vs_ndd_orders_kazan.xlsx')

In [ ]:
market_orders_df[
    market_orders_df['order_id'].isin(oid_set)
]

In [ ]:
for obj in sorted(
    sim.pickup_points,
    key=(lambda x: haversine.haversine((x.lat, x.lon), (pp.lat, pp.lon)))
):
    print(obj.id, len([d for d in obj.deliveries if d.delivered_flg]))

In [ ]:
print(sim.pickup_points_dict[1008].lat, sim.pickup_points_dict[1008].lon)
print(sim.pickup_points_dict[1002].lat, sim.pickup_points_dict[1002].lon)
print(sim.pickup_points_dict[1001].lat, sim.pickup_points_dict[1001].lon)
print(sim.pickup_points_dict[1004].lat, sim.pickup_points_dict[1004].lon)
print(sim.pickup_points_dict[1005].lat, sim.pickup_points_dict[1005].lon)